# Creator: Ivan Bardarov <br> (University of Strathclyde, March 2019)
## This is the mediator module which combines clothing detection, item collection,gram extraction, indexing and background removal

In [ ]:
import sys
sys.path.append('../recommender/')
import background_remover as br
from cloth_detector import *
import KNN_indexer as indexer
import gram_extractor as ge
import item_collector as ic
import uuid
from io import BytesIO
from os.path import exists
from os import remove, makedirs
from shutil import move
from pathlib import Path

# module setup
image_path_temp = Path("../static/tmp")
os.makedirs(image_path_temp, exist_ok=True)
learner, id2cat, cat2id = load_learner()
index_dict = {}

# create an dictionary with the all the existing index objects for faster retrieval
for cat in cat2id.values():
    grams_fn = ic.get_grams_filename(cat)
    if exists(grams_fn):
        data = ic.get_grams_collection(cat)
        index_dict[cat] = indexer.get_index(cat, data)

        
def get_unique_filename():
    """
    Get a unique file name for storing temporary images
    
    Returns
    -------
    str
        the path to the file
    """
    return f"{image_path_temp}/{uuid.uuid4()}.jpg"

def scale_bbox(bbox, res):
    """
    Fixes the coordinates order and scales it according to the original size
    
    Parameters
    ----------
    bbox : ndarray
        array with the relative bbox coordinates
    res : (int,int)
        the width and length of the original image
        
    Returns
    -------
    ndarray
        array with the scaled coordinates
    """
    h, w = res
    return np.array([bbox[1]*h, bbox[0]*w, bbox[3]*h, bbox[2]*w])

def get_recommendations(img_fn, cat):
    """
    Gets the top most similar items from the collection
    
    Parameters
    ----------
    img_fn : str
        the filename of the chosen image
    cat : int
        the id of the chosen category
        
    Returns
    -------
    ndarray
        array with top results
    """
    result = []
    if cat in index_dict:
        #remove the background and get the gram matrix
        br.remove_background(img_fn).save(img_fn)
        features_vector = ge.get_gram(img_fn)
        
        #get the most similar items
        ids = indexer.get_knn(index_dict[cat], features_vector, k=4)[0]
        
        #get the filenames of the items to be returned
        ar = ic.get_filenames_collection(cat)
        for i in ids:
            result.append(ar[i])
    return result

def get_detected_items(img):
    """
    Hit the detection model to get all the clothes on an input photo
    
    Parameters
    ----------
    img : Image
        the input image for the object detection model
        
    Returns
    -------
    i_fns: ndarray
        array with the filenames of the detected items
    cat_list: ndarray
        array with the categories of the detected items
    id2cat: ndarray
        a list with all the names of all the categories
    """
    # save the received image
    org_fn = get_unique_filename()
    img.save(org_fn)

    i_fns = []
    cat_list = []
    
    #make a prediction on the image
    bboxes, cats, _ = predict(learner, org_fn)
    
    #handle the results
    for cat, bbox in zip(cats,bboxes):
        # save and crop each of the detected items
        i_fn = get_unique_filename()
        bbox = scale_bbox(bbox, img.size)
        img.crop(tuple(bbox)).save(i_fn)
        
        #add to the results to be returned
        i_fns.append(i_fn)
        cat_list.append(cat)
    
    return i_fns, cat_list, id2cat

In [1]:
def get_largest_index(scores):
    return scores.max(0)[1]

def get_largest_bbox(bboxes, largest_index, img_res):
    bb = bboxes[largest_index].squeeze()
    bb = scale_bbox(bb, img_res)
    return bb

def get_results(fn, cat):
    gram = ge.get_gram(fn)
    return get_knn(index, act)[1]